In [1]:
#This script can be used to generate the database which feeds into the uncertainty model.

#Author: Eddy

In [2]:
import pandas as pd
import numpy as np
import scml
import sys
from typing import List, Union
from scml.scml2020 import SCML2020Agent
from scml.scml2020.agents import DecentralizingAgent, BuyCheapSellExpensiveAgent, RandomAgent, DoNothingAgent

In [3]:
#Add whatever folder the generatedata.py script is in on your computer.
#generatedata.py has to be edited so that generate_world() takes in n_processes as a parameter.
#This has to be run on the environment where SCML and NEGMAS packages were imported

sys.path.append(r'C:/FILL IN HERE.../SCML/scml2020/myagent')
import generatedata as gd

In [4]:
def produce_game_log(T,L,full_log_df):

    cols = set(full_log_df.columns)

    cols_to_keep = set()
    for i in range(L):
        cols_to_keep.add('sold_quantity_' + str(i))
        cols_to_keep.add('unit_price_' + str(i))

    drop_cols = cols - cols_to_keep

    #create a column for the timestep
    #should this start from 0 or 1?
    t = []
    for i in range(T):
        t.append(str(i))

    # Select only the data that we need for this computation
    y = full_log_df.drop(drop_cols, axis = 1)
    y.insert(0,'t',t)
    return y

In [5]:
def vertical_concat(list_of_logs):
    df = list_of_logs[0]
    game_index_column = [1] * len(df.index)
    
    for i in range(1,len(list_of_logs)):
        df = pd.concat([df, list_of_logs[i]], ignore_index=True)
        temp_game_index_column = [i+1] * len(list_of_logs[i].index)
        game_index_column.extend(temp_game_index_column)
        
    #print((game_index_column))    
    df.insert(0,'game_index', game_index_column)
    return df

In [9]:
def generate_model_input(n_steps: int, competitors: List[Union[str, SCML2020Agent]], n_agents_per_competitor, n_processes):
    """Takes in inputs and generates three game logs' worth of relevant game info according to this input. This
    can be fed into the uncertainty prediction model.
    
    Args:
        n_steps: Integer containing how many time steps each game will have.
        competitors: A list of the agents to be included in the simulation. For instance, 
            COMPETITORS = [DecentralizingAgent, BuyCheapSellExpensiveAgent, RandomAgent].
        n_agents_per_competitor: Integer representing how many agents each competitor will have.
        n_processes: Integer representing how many levels of inventory each game should have. Ranges from 4 to 6. 
                     Inventory index starts from 0.
        
    Returns:
        Pandas dataframe which has the three game logs' worth of relevant game info.
        
    """
    #1. generate full worlds with all data (number of logs is number of agents * 20)
    worlds_df_list = [];
    for j in range(20):
        worlds = gd.generate_world(n_steps,
                            COMPETITORS,
                            n_agents_per_competitor,
                            n_processes - 1)
        for i, world in enumerate(worlds):
            world.run()
            worlds_df_list.append(world.stats_df)
    
    #2. produce game logs of relevant information from full world data
    df_logs_list = []
    for worlds_df in worlds_df_list:
        df_logs_list.append(produce_game_log(n_steps,10,worlds_df))
    
    #3. vertically concatenate all the game logs together
    return vertical_concat(df_logs_list)

In [10]:
#The competitor agents can be edited here. Then run generate_model_input.
COMPETITORS = [DecentralizingAgent, BuyCheapSellExpensiveAgent, RandomAgent, DoNothingAgent]
df = generate_model_input(10, COMPETITORS, 1, 5)
df

,game_index,t,sold_quantity_0,unit_price_0,sold_quantity_1,unit_price_1,sold_quantity_2,unit_price_2,sold_quantity_3,unit_price_3,sold_quantity_4,unit_price_4
0,1,0,20,10.0,0,NaN,0,NaN,0,NaN,0,NaN
1,1,1,20,10.0,0,NaN,0,NaN,0,NaN,0,NaN
2,1,2,20,10.0,0,NaN,0,NaN,0,NaN,0,NaN
3,1,3,20,10.0,19,32.368421,0,NaN,0,NaN,0,NaN
4,1,4,20,10.0,0,NaN,0,NaN,0,NaN,20,56.0
5,1,5,20,10.0,18,34.444444,0,NaN,0,NaN,20,56.0
6,1,6,0,NaN,4,37.000000,0,NaN,0,NaN,20,56.0
7,1,7,0,NaN,50,35.660000,0,NaN,0,NaN,20,56.0
8,1,8,0,NaN,2,37.000000,0,NaN,0,NaN,20,56.0
9,1,9,0,NaN,10,34.800000,0,NaN,0,NaN,20,56.0


In [8]:
#uncomment this if you want to output this log to csv
#df.to_csv("INSERT FILE PATH HERE")